In [1]:
import numpy as np
import pandas as pd

In [2]:
ddos = pd.read_csv('DDos_2m.csv')

In [3]:
ddos.columns

Index(['Unnamed: 0', 'FlowDuration', 'TotalFwdPackets', 'TotalBackwardPackets',
       'TotalLengthofFwdPackets', 'TotalLengthofBwdPackets',
       'FwdPacketLengthMax', 'FwdPacketLengthMin', 'FwdPacketLengthMean',
       'FwdPacketLengthStd', 'BwdPacketLengthMax', 'BwdPacketLengthMin',
       'BwdPacketLengthMean', 'BwdPacketLengthStd', 'FlowBytes/s',
       'FlowPackets/s', 'FlowIATMean', 'FlowIATStd', 'FlowIATMax',
       'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd', 'FwdIATMax',
       'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd', 'BwdIATMax',
       'BwdIATMin', 'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags', 'BwdURGFlags',
       'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets/s', 'BwdPackets/s',
       'MinPacketLength', 'MaxPacketLength', 'PacketLengthMean',
       'PacketLengthStd', 'PacketLengthVariance', 'FINFlagCount',
       'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount', 'ACKFlagCount',
       'URGFlagCount', 'CWEFlagCount', 'ECEFlagCount', 'Down/UpRatio

In [4]:
ddos = ddos.drop(columns = ['Unnamed: 0'])

In [5]:
ddos.shape

(2011542, 78)

In [6]:
ddos['Label'].value_counts()

MSSQL      578745
Syn        428475
UDP        375468
NetBIOS    365749
LDAP       191512
BENIGN      52897
Portmap     18696
Name: Label, dtype: int64

In [7]:
df = ddos.copy()
cat_columns = df.select_dtypes(['object']).columns
cat_columns

Index(['Label'], dtype='object')

In [8]:
df[cat_columns] = df[cat_columns].astype('category')
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
df
ddos = df
ddos.dtypes

FlowDuration               float64
TotalFwdPackets            float64
TotalBackwardPackets       float64
TotalLengthofFwdPackets    float64
TotalLengthofBwdPackets    float64
                            ...   
IdleMean                   float64
IdleStd                    float64
IdleMax                    float64
IdleMin                    float64
Label                         int8
Length: 78, dtype: object

In [9]:
X = ddos.iloc[:, 0:-1]
y = ddos.iloc[:, -1]

In [10]:
ddos = None; del ddos

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [12]:
X_train.to_csv('X_train_2m.csv')

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
from keras.utils import np_utils
X_train=np.expand_dims(X_train,axis=2)
X_test=np.expand_dims(X_test,axis=2)
X.shape

(2011542, 77)

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM
from tensorflow.keras.utils import to_categorical

In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [16]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(77,1)))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
from keras.callbacks import EarlyStopping
import time

start = time.time()
print("Timer Begin")

model.fit(X_train,Y_train,batch_size=30,epochs=1,
          validation_split=0, callbacks=[EarlyStopping(monitor='loss',patience=2)])

end = time.time()
print(end - start)

Timer Begin
44925/44925 [==============================] - 141s 3ms/step - loss: 0.1233 - accuracy: 0.9653
141.61586594581604


In [18]:
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)

In [19]:
rf_conf_mat = confusion_matrix(Y_test, y_pred)
print("Transfer learning confusion matrix: \n", rf_conf_mat)

Transfer learning confusion matrix: 
 [[ 17427      0      0     39      0     38      0]
 [     1  62360    693      8      0      6      1]
 [    11   4449 180003     17      0      6   7002]
 [    14      0     21 120094      0      2    255]
 [    13      0     25   6099      0     14     13]
 [   111      0    135      1      0 141105     23]
 [     1      0   2646      9      0      1 121166]]


In [20]:
CR = classification_report(Y_test,y_pred,digits= 4)
print(CR)

D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0     0.9914    0.9956    0.9935     17504
           1     0.9334    0.9888    0.9603     63069
           2     0.9808    0.9400    0.9600    191488
           3     0.9511    0.9976    0.9738    120386
           4     0.0000    0.0000    0.0000      6164
           5     0.9995    0.9981    0.9988    141375
           6     0.9432    0.9785    0.9606    123823

    accuracy                         0.9674    663809
   macro avg     0.8285    0.8427    0.8353    663809
weighted avg     0.9591    0.9674    0.9629    663809



D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
from keras.callbacks import EarlyStopping
import time

start = time.time()
print("Timer Begin")

model.fit(X_train,Y_train,batch_size=30,epochs=1,
          validation_split=0, callbacks=[EarlyStopping(monitor='loss',patience=2)])

end = time.time()
print(end - start)

Timer Begin
44925/44925 [==============================] - 143s 3ms/step - loss: 0.1121 - accuracy: 0.9701
143.75908946990967


In [22]:
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)

In [23]:
rf_conf_mat = confusion_matrix(Y_test, y_pred)
print("Transfer learning confusion matrix: \n", rf_conf_mat)

Transfer learning confusion matrix: 
 [[ 17373      0      5     34      1     76     15]
 [     1  61970   1084      8      0      5      1]
 [     9   4196 184342     18      0      5   2918]
 [    15      0     21 120174      0      2    174]
 [    14      0     25   6098      4     13     10]
 [    79      0    135      1      0 141137     23]
 [     1      0   2656     10      0      1 121155]]


In [24]:
CR = classification_report(Y_test,y_pred, digits= 4)
print(CR)

              precision    recall  f1-score   support

           0     0.9932    0.9925    0.9929     17504
           1     0.9366    0.9826    0.9590     63069
           2     0.9791    0.9627    0.9708    191488
           3     0.9512    0.9982    0.9741    120386
           4     0.8000    0.0006    0.0013      6164
           5     0.9993    0.9983    0.9988    141375
           6     0.9747    0.9785    0.9766    123823

    accuracy                         0.9734    663809
   macro avg     0.9477    0.8448    0.8391    663809
weighted avg     0.9722    0.9734    0.9689    663809



In [25]:
from keras.callbacks import EarlyStopping
import time

start = time.time()
print("Timer Begin")

model.fit(X_train,Y_train,batch_size=30,epochs=1,
          validation_split=0, callbacks=[EarlyStopping(monitor='loss',patience=2)])

end = time.time()
print(end - start)

Timer Begin
44925/44925 [==============================] - 143s 3ms/step - loss: 0.1110 - accuracy: 0.9711
142.91303062438965


In [26]:
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)

In [27]:
rf_conf_mat = confusion_matrix(Y_test, y_pred)
print("Transfer learning confusion matrix: \n", rf_conf_mat)

Transfer learning confusion matrix: 
 [[ 17421      0      0     69      3     10      1]
 [     0  62313    743      8      2      3      0]
 [     9   4306 180142     21      1      3   7006]
 [    12      0     21 120116      0      2    235]
 [    14      0     25   5782    328      3     12]
 [   139      0    135      1     40 141037     23]
 [     1      0   2630      9      1      0 121182]]


In [28]:
CR = classification_report(Y_test,y_pred, digits= 4)
print(CR)

              precision    recall  f1-score   support

           0     0.9901    0.9953    0.9926     17504
           1     0.9354    0.9880    0.9610     63069
           2     0.9807    0.9407    0.9603    191488
           3     0.9533    0.9978    0.9750    120386
           4     0.8747    0.0532    0.1003      6164
           5     0.9999    0.9976    0.9987    141375
           6     0.9434    0.9787    0.9607    123823

    accuracy                         0.9680    663809
   macro avg     0.9539    0.8502    0.8498    663809
weighted avg     0.9678    0.9680    0.9641    663809

